In [22]:
from scipy.optimize import minimize

In [23]:
def calc_mp(fv,c,m,ytm):
    return fv*(ytm**-m) + sum(c*(ytm**-i) for i in range(1,m+1))

def calc_ytm(fv,c,m,p):
    c*=fv/100
    def obj_f(x):
        return (p-calc_mp(fv,c,m,x))**2
    sol = minimize(obj_f,1.05)
    return (sol.x[0]-1)*100

In [24]:
args_list = [
    (1000,4.375,5,929.55),
    (1000,4.875,6,946.68),
    (50000,7.25,4,52167.69),
    (1000,3.735,7,871.56),
    (1000,8.25,3,1062.14),
    (1000,5,10,1050),
]
for a in args_list:
    ytm = calc_ytm(*a)
    print(f'{a} {ytm:.3f}%')

(1000, 4.375, 5, 929.55) 6.050%
(1000, 4.875, 6, 946.68) 5.958%
(50000, 7.25, 4, 52167.69) 5.999%
(1000, 3.735, 7, 871.56) 6.039%
(1000, 8.25, 3, 1062.14) 5.928%
(1000, 5, 10, 1050) 4.372%


In [25]:
import pandas as pd
import numpy as np

In [63]:
def new_corr_coef(s_x, s_y):
    """
    s_x: pd.Series, np.array
    """
    assert len(s_x)==len(s_y)
    n = len(s_x)
    df = pd.DataFrame({'x':s_x, 'y':s_y})
    df = df.sort_values(by='x', ascending=True)
    rank = df['y'].rank(method='min')
    coef = 1 - 3*sum(abs(rank-rank.shift(1))[1:])/(n**2-1)
    return coef

def rank_y(df):
    sorted_df = df.sort_values(by='y')
    ranked_series = sorted_df.groupby('y').cumcount() + 1
    return ranked_series

def new_corr_coef_2(s_x, s_y):
    """
    s_x: pd.Series, np.array
    """
    assert len(s_x)==len(s_y)
    n = len(s_x)
    df = pd.DataFrame({'x':s_x, 'y':s_y})
    df = df.sort_values(by='x', ascending=True)
    rank = df['y'].rank(method='min')
    rank2 = df['y'].rank(method='max')
    coef = 1 - n*sum(abs(rank-rank.shift(1))[1:])/2/sum(rank2*(n-rank2))
    return coef


In [53]:
from scipy.stats import bernoulli

In [66]:
a = bernoulli(.4)
a = a.rvs(100000)
b = bernoulli(.5)
b = b.rvs(100000)
print(new_corr_coef(a,b))
print(new_corr_coef_2(a,b))

0.25464422072546444
0.001699963718951869
